In [40]:
from tikapi import TikAPI, ValidationException, ResponseException
import json
import glob
import os
import re
import pandas as pd

api = TikAPI("1jDHBpHXtsycrmEqK5AVvhvPZq3N2ZGeFNJKzYEJ2MHTPb4y")
User = api.user(accountKey="6EjyV4OptbvUz1cKgMTnvbXNH1wKPqLVV42jKlzydbVrFLnb")

_data_path = "./data/"
video_ids_file = _data_path + "video_list.csv"

max_requests_for_each_video = 20
number_of_comment_each_request = 30 #(max possible value is 30)

# Load video IDs

In [42]:
df = pd.read_csv(video_ids_file)

# extract video ids from links
df["id"] = [re.sub("video/", "", re.findall("video/[0-9]{19}", link)[0]) for link in df["Link"]]
print(f"First 5 video ids:\n{df.head(5)['id']}")


First 5 video ids:
0    7144986246215929094
1    7146104387079900421
2    7138834940187151622
Name: id, dtype: object


In [67]:

j = 0 #single video request counter
total_n_comments_read = 0 #total comments counter

# iterate over videos
for i, row in df.iterrows():
    
    # check if comments to this video have already been downloaded
    if not row['Done']:
        
        id = row['id']
        file = _data_path + row['Politician'] + "_com_" + id + ".json"
        comments = [] #it will contain all the json responses for each request
        
        print("\n#########################################\nProcessing video",id)

        try:
            response = User.posts.comments.list(
                media_id = video_id,
                count = number_of_comment_each_request # (max possible value is 30)
            )

            response_json = response.json()
            print(response_json)
            
            j=1
            while(response):    
                print(response_json)
                # append response data to result list
                comments.append(response_json['comments'])
                total_n_comments_read += len(response_json['comments'])
                    
                # check if we reached maximum n of requests for this video
                if j >= max_requests_for_each_video:
                    break #stop
                    
                # if not, do next request
                cursor = response.json().get('cursor')
                response = response.next_items()  

                if response == None:
                    break # no more responses

                response_json = response.json()
                
                if response_json['comments'] == None:
                    break #no more comments (?)
                j+=1

        except ValidationException as e:
            print(e, e.field)

        except ResponseException as e:
            print(e, e.response.status_code)

        # remove comments from last response object. Now, only the meta data remains.
        response_json['comments'] = None

        # creat dictionary with metadata and comments
        out = {"meta": response_json, "comments": comments}

        with open(file, 'w') as f:
            json.dump(out, f)




#########################################

Processing video 7144986246215929094


#########################################

Processing video 7146104387079900421


#########################################

Processing video 7138834940187151622


# Example for reading data

In [69]:
with open("./data/Salvini_com_7146104387079900421.json") as infile:
    # returns JSON object as 
    # a dictionary
    data = json.load(infile)
data

{'meta': [], 'comments': []}

<p style="text-align: center;"> <i><b> Fin </b></i> </p>